<a href="https://colab.research.google.com/github/abhibrat/Bigram-Language-Model/blob/main/Bigram_Neural_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [3]:
chars = list(sorted(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos = {i:s for s,i in stoi.items()}

In [12]:
import torch
import torch.nn.functional as F

In [46]:
xs = []
ys = []
for w in words[:1]:
  chars = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chars, chars[1:]):
    idx1 = stoi[ch1]
    idx2 = stoi[ch2]

    xs.append(idx1)
    ys.append(idx2)

    print(ch1, ch2)



. e
e m
m m
m a
a .


In [48]:
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [24]:
xenc = F.one_hot(xs, num_classes=27).float()

In [18]:
W = torch.randn([27,27])
W.shape

torch.Size([27, 27])

In [21]:
W[0]

tensor([-0.1659, -0.3569, -2.2936, -0.3436,  0.2283, -0.7734, -1.0048, -0.1006,
         0.0287, -0.7943,  0.5693,  1.1133,  0.0396, -0.7899,  0.6077,  0.5158,
        -0.6399,  0.8918,  1.4564,  1.6335, -0.2923, -0.1900,  0.4007, -2.0067,
        -2.1143, -0.1030, -0.2867])

In [38]:
logits = xenc @ W
logits

tensor([[-0.1659, -0.3569, -2.2936, -0.3436,  0.2283, -0.7734, -1.0048, -0.1006,
          0.0287, -0.7943,  0.5693,  1.1133,  0.0396, -0.7899,  0.6077,  0.5158,
         -0.6399,  0.8918,  1.4564,  1.6335, -0.2923, -0.1900,  0.4007, -2.0067,
         -2.1143, -0.1030, -0.2867],
        [-0.1192, -0.0891,  0.2304,  1.3833, -0.1218, -0.5129, -0.2321,  1.2575,
          0.2231, -0.8001, -0.3109, -0.8796,  2.0067, -0.7737,  2.2341, -0.5573,
         -1.2147,  0.7093, -1.8323, -0.2658, -2.1418, -1.0662,  0.0774,  1.5379,
          1.0650,  0.7472,  1.4222],
        [ 0.3978,  0.0080, -0.0043, -0.0834, -2.0472, -0.4544,  0.9618, -0.7746,
          0.5157,  0.7805, -0.6160,  1.1989, -0.3088,  0.6144, -1.3606,  1.3996,
         -0.8041,  1.1498, -1.4748,  0.0264, -0.9099,  0.2048, -0.5033, -0.4294,
          0.4746,  0.9195, -0.4502],
        [ 0.3978,  0.0080, -0.0043, -0.0834, -2.0472, -0.4544,  0.9618, -0.7746,
          0.5157,  0.7805, -0.6160,  1.1989, -0.3088,  0.6144, -1.3606,  1.3996

In [39]:
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)

In [40]:
probs[0].sum()

tensor(1.)

In [44]:
a = probs[torch.arange(5), ys]
a

tensor([0.0136, 0.0089, 0.0523, 0.0285, 0.0071])

In [45]:
torch.arange(5)

tensor([0, 1, 2, 3, 4])

In [49]:
nlls = torch.zeros(5)
for i in range(5):
  idx1 = xs[i].item()
  idx2 = ys[i].item()
  print(f'Bigram example {i+1}: {itos[idx1]} {itos[idx2]}')
  p = probs[i,idx2]
  print(f'Probability of label: {itos[idx2]} --> {p}')
  log_likelihood = p.log()
  print(f'{log_likelihood=}')
  negative_log_likelihood = -log_likelihood
  print(f'{negative_log_likelihood=}')
  nlls[i] = negative_log_likelihood

print('======')
print('Average negative log likelihood i.e loss = ', nlls.mean().item())

Bigram example 1: . e
Probability of label: e --> 0.013572633266448975
log_likelihood=tensor(-4.2997)
negative_log_likelihood=tensor(4.2997)
Bigram example 2: e m
Probability of label: m --> 0.008852507919073105
log_likelihood=tensor(-4.7271)
negative_log_likelihood=tensor(4.7271)
Bigram example 3: m m
Probability of label: m --> 0.0522775799036026
log_likelihood=tensor(-2.9512)
negative_log_likelihood=tensor(2.9512)
Bigram example 4: m a
Probability of label: a --> 0.028507491573691368
log_likelihood=tensor(-3.5576)
negative_log_likelihood=tensor(3.5576)
Bigram example 5: a .
Probability of label: . --> 0.007137717213481665
log_likelihood=tensor(-4.9424)
negative_log_likelihood=tensor(4.9424)
Average negative log likelihood i.e loss =  4.095578193664551


In [50]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [78]:
# Forward Pass
logits = xenc @ W
counts = logits.exp()
probs = counts/counts.sum(1, keepdim=True)
loss = -probs[torch.arange(5), ys].log().mean()
loss.data

tensor(0.2870)

In [76]:
W.grad = None
loss.backward()

In [77]:
W.data += -1 * W.grad

In [87]:
# Output after training on the first word
for i in range(5):
  out = []
  idx = 0
  while True:
    x_enc = F.one_hot(torch.tensor(idx), num_classes=27).float()
    # print(f'{x_enc=}')
    logits = x_enc @ W
    counts = logits.exp()
    # print(f'{counts=}')
    probs = counts/counts.sum()

    idx = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[idx])
    if idx == 0:
      break

  print(''.join(out))




ema.
emma.
ema.
emmmma.
emmmma.


In [122]:
# Considering all words

# Preapare dataset
xs = []
ys = []
for w in words:
  chars = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chars, chars[1:]):
    idx1 = stoi[ch1]
    idx2 = stoi[ch2]

    xs.append(idx1)
    ys.append(idx2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('Number of elements: ', num)

#Input to the neuron
xenc = F.one_hot(xs, num_classes=27).float()

# Initialize neuron weights
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)


Number of elements:  228146


In [123]:
# Generation before training
for i in range(20):
  out = []
  idx = 0
  while True:
    x_enc = F.one_hot(torch.tensor(idx), num_classes=27).float()
    # print(f'{x_enc=}')
    logits = x_enc @ W
    counts = logits.exp()
    # print(f'{counts=}')
    probs = counts/counts.sum()

    idx = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[idx])
    if idx == 0:
      break

  print(''.join(out))

yofvwqjkdktcnblz.
yw.
ovrzwchmbqdaydgqfbmhqpouywytozbzwzrzqrdj.
ytdkkv.
ztsmltgpcxtikfblqnsastwzhtc.
onmkisedqppxwwluxin.
yddlpfkdjjljagkzisifbwfbjmcnbyguiaxwvyyg.
dphbjmhyqsedtoziszhksufzwytkadicmoufbmwmpqqfbmrddjucvknikatviyxvtjxhufblt.
ttjbmafplwgmhgqbpokalv.
pxbqrilpuslpalphhuxh.
ozwwchufphkwmgpcjieaklwmhdztphphqdtiarincszrmbzt.
andjufoufaumktwgwpvkfr.
zwtkt.
ozitdpsicxgkokdwcpfbqaomyhpchyddfbc.
s.
phkvm.
ekt.
gmdtkpafrciirhcgksxralpjzxszwtrmhusbmkdajdglpchuszfpoalmbmpejmfjbgluzoztduepxtjszxepxxekkmgwxerzxr.
tkc.
t.


In [119]:
# Training
for i in range(50):
  # Forward Pass
  logits = xenc @ W
  counts = logits.exp()
  probs = counts/counts.sum(1, keepdim=True)
  loss = -probs[torch.arange(num), ys].log().mean()
  print(loss.data.item())

  # Backprop
  W.grad = None
  loss.backward()

  # Gradient Descent
  W.data += -10 * W.grad


3.1797327995300293
3.147189140319824
3.117339849472046
3.0898513793945312
3.064443349838257
3.0408785343170166
3.0189592838287354
2.998518705368042
2.9794163703918457
2.961529493331909
2.9447529315948486
2.928992509841919
2.9141643047332764
2.900193929672241
2.887012481689453
2.874558448791504
2.862776279449463
2.851613998413086
2.841024160385132
2.830965757369995
2.8213977813720703
2.812286138534546
2.8035967350006104
2.7953009605407715
2.7873711585998535
2.7797818183898926
2.7725107669830322
2.7655372619628906
2.7588419914245605
2.7524075508117676
2.7462172508239746
2.7402572631835938
2.7345142364501953
2.7289750576019287
2.7236287593841553
2.7184643745422363
2.713472366333008
2.708644151687622
2.7039716243743896
2.699446678161621
2.695061683654785
2.6908113956451416
2.6866886615753174
2.682687759399414
2.678804397583008
2.675032138824463
2.671367883682251
2.6678061485290527
2.6643431186676025
2.6609747409820557


In [138]:
# Generation after training
for i in range(20):
  out = []
  idx = 0
  while True:
    x_enc = F.one_hot(torch.tensor(idx), num_classes=27).float()
    # print(f'{x_enc=}')
    logits = x_enc @ W
    counts = logits.exp()
    # print(f'{counts=}')
    probs = counts/counts.sum()

    idx = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[idx])
    if idx == 0:
      break

  print(''.join(out))

mxtrabyna.
bjanaiech.
kiken.
shan.
so.
kadyly.
zaisionch.
ti.
sh.
gfinahaxatana.
em.
kesaleyna.
aran.
lioen.
erorraror.
jennlowonnde.
anien.
jade.
jon.
dastan.


In [139]:
# Training with regularization
for i in range(10):
  # Forward Pass
  logits = xenc @ W
  counts = logits.exp()
  probs = counts/counts.sum(1, keepdim=True)
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.data.item())

  # Backprop
  W.grad = None
  loss.backward()

  # Gradient Descent
  W.data += -1 * W.grad

2.4913740158081055
2.491368293762207
2.491363286972046
2.4913580417633057
2.4913527965545654
2.491346836090088
2.4913418292999268
2.4913365840911865
2.4913313388824463
2.491325616836548
